## Homework 5
Assignment Group 12  
Ke Xu  
FangZhou Liu

## Problem 1

## split the data to training set and test set

In [15]:
import pandas as pd, numpy as np
import re
import nltk

df = pd.read_csv('imdb_master.csv')
X = df['review']
y = df['label']
#split the data to training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

## count all words and to find top 2500

In [5]:
# count all words and find the top 2500 words
data = ''
for i in X_train:
    data = data + " " + i
cleandata =re.sub('[^A-Za-z]+', ' ', data.lower())
datalist = cleandata.split()
fdist2 = nltk.FreqDist(datalist)
words = fdist2.most_common(2500)

## split each review to list

In [9]:
X_train = list(X_train.values.flatten())
y_train = list(y_train.values.flatten())
for i in range(len(X_train)):
    X_train[i] = re.sub('[^A-Za-z]+', ' ', X_train[i].lower()).split()

## define a function to generate features

In [10]:
#generate feature matrix, for each reviews if the word in the review then label it to 1
def generateX(X_re):
    result = np.zeros(len(words))
    for l in X_re:
        x1 = np.zeros(len(words))
        for i in range(len(words)):
            if words[i][0] in l:
                x1[i] = 1
        result = np.column_stack((result, x1))
    return result.transpose()

## generate features from X_train

In [23]:
train_X = generateX(X_train)
#delet the first row, since the first row is added by generateX function(which is all 0)
train_X = np.delete(train_X,0,0)
print('The features we generate from the first review of X_train:')
print('length:',len(train_X[0]))
print(train_X[0])

The features we generate from the first review of X_train:
length: 2500
[1. 1. 1. ... 0. 0. 0.]


## since MaxEnt is the same model as logistic regression, we use skleaner to train our model

## use the feature matrix we get from X_train to train our model

In [12]:
#training our model, the MaxEnt is same as Multinomial logistic regression model.
from sklearn.linear_model import LogisticRegressionCV
#training our model with cross-validation = 5
clf = LogisticRegressionCV(cv=5, random_state=0, multi_class='multinomial')
clf.fit(train_X, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=0, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

## generate features from X_test

In [16]:
#prepare our test dataset
X_test = list(X_test.values.flatten())
y_test = list(y_test.values.flatten())
for i in range(len(X_test)):
    X_test[i] = re.sub('[^A-Za-z]+', ' ', X_test[i].lower()).split()

In [17]:
test_X = generateX(X_test)
test_X = np.delete(test_X,0,0)

## Put the feature matrix which from X_test in our classifier to do prediction, and print the accuracy

In [19]:
#test our model and out the accuracy
final = clf.predict(test_X)
print('Accuracy:')
print(clf.score(test_X,y_test))

Accuracy:
0.8313333333333334


There are some words can affect the accuracy, such as 'is, the , are' and so on. Those words appear in both positive reviews and negative reviews. And those world must appear in top 2500 words list. This will affect accuracy. For improvment, we can find top 2500 words without those words to train our model. And also we can increase the size of words list, that also may improve the accuracy.

## Problem 2

In [52]:
import numpy as np
import random

In [53]:
states = ('Hot', 'Cold')
end_state = 'End'
 
observations1 = ('3','3','1','1','2','2','3','1','3')
observations2 = ('3','3','1','1','2','2','3','1','2')

start_probability = {'Hot': 0.8, 'Cold': 0.2}

transition_probability = {
   'Hot'  : {'Hot': 0.6, 'Cold': 0.3, 'End': 0.1},
   'Cold' : {'Hot': 0.4, 'Cold': 0.5, 'End': 0.1},
   }
 
emission_probability = {
   'Hot'  : {'1': 0.2, '2': 0.4, '3': 0.4},
   'Cold' : {'1': 0.5, '2': 0.4, '3': 0.1},
   }

Foward & Backard

In [83]:
# fwd_bkw function include both Forward Algorithm and Backward Algorithm
def fwd_bkw(y, states, a_0, a, e, end_st):
    L = len(y)
    fwd = []
    f_prev = {}
    # forward part of the algorithm
    for i, y_i in enumerate(y):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = a_0[st]
            else:
                prev_f_sum = sum(f_prev[k]*a[k][st] for k in states)
 
            f_curr[st] = e[st][y_i] * prev_f_sum
 
        fwd.append(f_curr)
        f_prev = f_curr
 
    p_fwd=sum(f_curr[k]*a[k][end_st] for k in states)
    p_fwd_list = fwd

    bkw = []
    b_prev = {}
    # backward part of the algorithm
    for i, y_i_plus in enumerate(reversed(y[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = a[st][end_st]
            else:
                b_curr[st] = sum(a[st][l]*e[l][y_i_plus]*b_prev[l] for l in states)
 
        bkw.insert(0,b_curr)
        b_prev = b_curr
 
    p_bkw = sum(a_0[l] * e[l][y[0]] * b_curr[l] for l in states)
    bkw_list= bkw
    # merging the two parts
    posterior = []
    for i in range(L):
        posterior.append({st: fwd[i][st]*bkw[i][st]/p_fwd for st in states})
        
 
    
    return p_fwd, p_bkw, p_fwd_list, bkw_list, posterior

In [85]:
def example(observations):
    return fwd_bkw(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)

implement Forward Algorithm & Backward Algorithm for 2 obserations

In [88]:
fwd, bkw, fwd_list, bkw_list,posterior = example(observations1)
print('Foreword:')
print(fwd_list)
print('Backword:')
print(bkw_list)
print('Posterior:')
print(posterior)

Foreword:
[{'Hot': 0.32000000000000006, 'Cold': 0.020000000000000004}, {'Hot': 0.08000000000000002, 'Cold': 0.010600000000000002}, {'Hot': 0.010448000000000002, 'Cold': 0.014650000000000003}, {'Hot': 0.0024257600000000008, 'Cold': 0.005229700000000001}, {'Hot': 0.0014189344000000003, 'Cold': 0.0013370312000000003}, {'Hot': 0.0005544692480000001, 'Cold': 0.00043767836800000015}, {'Hot': 0.00020310115840000006, 'Cold': 3.851799584000002e-05}, {'Hot': 2.7453578675200012e-05, 'Cold': 4.009467272000001e-05}, {'Hot': 1.3004006517248004e-05, 'Cold': 2.828340996256001e-06}]
Backword:
[{'Hot': 4.71096613296e-06, 'Cold': 3.7862794401600007e-06}, {'Hot': 1.6938878256000002e-05, 'Cold': 2.1521178384000004e-05}, {'Hot': 5.5919044800000004e-05, 'Cold': 6.819061920000001e-05}, {'Hot': 0.00020839824, 'Cold': 0.00020607504000000004}, {'Hot': 0.000588564, 'Cold': 0.000559524}, {'Hot': 0.0017559, 'Cold': 0.0013929000000000003}, {'Hot': 0.00639, 'Cold': 0.007410000000000002}, {'Hot': 0.027, 'Cold': 0.0210

In [89]:
fwd, bkw, fwd_list, bkw_list,posterior = example(observations2)
print('Foreword:')
print(fwd_list)
print('Backword:')
print(bkw_list)
print('Posterior:')
print(posterior)

Foreword:
[{'Hot': 0.32000000000000006, 'Cold': 0.020000000000000004}, {'Hot': 0.08000000000000002, 'Cold': 0.010600000000000002}, {'Hot': 0.010448000000000002, 'Cold': 0.014650000000000003}, {'Hot': 0.0024257600000000008, 'Cold': 0.005229700000000001}, {'Hot': 0.0014189344000000003, 'Cold': 0.0013370312000000003}, {'Hot': 0.0005544692480000001, 'Cold': 0.00043767836800000015}, {'Hot': 0.00020310115840000006, 'Cold': 3.851799584000002e-05}, {'Hot': 2.7453578675200012e-05, 'Cold': 4.009467272000001e-05}, {'Hot': 1.3004006517248004e-05, 'Cold': 1.1313363985024004e-05}]
Backword:
[{'Hot': 7.2357114412800015e-06, 'Cold': 5.815469450880002e-06}, {'Hot': 2.6016873408000007e-05, 'Cold': 3.305539411200001e-05}, {'Hot': 8.588384640000002e-05, 'Cold': 0.00010473874560000003}, {'Hot': 0.00032000832, 'Cold': 0.00031655232000000013}, {'Hot': 0.0009033120000000001, 'Cold': 0.0008601120000000002}, {'Hot': 0.0026892, 'Cold': 0.0021492000000000004}, {'Hot': 0.00972, 'Cold': 0.011880000000000002}, {'Hot

Viterbi

In [76]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}
 
    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
 
    # Run Viterbi for t > 0
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
 
        for y in states:
            (prob, state) = max((V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states)
            V[t][y] = prob
            newpath[y] = path[state] + [y]
 
        # Don't need to remember the old paths
        path = newpath
     
    #print_dptable(V)
    (prob, state) = max((V[t][y], y) for y in states)
    return (prob, path[state],V)



prob1, p_hidden1,v1 = viterbi(observations1,states,start_probability,transition_probability,emission_probability)
prob2, p_hidden2,v2 = viterbi(observations2,states,start_probability,transition_probability,emission_probability)
# assess accuracy of the HMM model

print("Viterbi1: ",prob1)
print("path1: ",p_hidden1)
print(v1)
print()
print("Viterbi2: ",prob2)
print("path2: ",p_hidden2)
print(v2)

Viterbi1:  7.644119040000002e-07
path1:  ['Hot', 'Hot', 'Cold', 'Cold', 'Hot', 'Hot', 'Hot', 'Hot', 'Hot']
[{'Hot': 0.32000000000000006, 'Cold': 0.020000000000000004}, {'Hot': 0.07680000000000002, 'Cold': 0.009600000000000003}, {'Hot': 0.009216000000000002, 'Cold': 0.011520000000000002}, {'Hot': 0.0011059200000000003, 'Cold': 0.0028800000000000006}, {'Hot': 0.00046080000000000014, 'Cold': 0.0005760000000000001}, {'Hot': 0.00011059200000000003, 'Cold': 0.00011520000000000004}, {'Hot': 2.6542080000000007e-05, 'Cold': 5.7600000000000024e-06}, {'Hot': 3.1850496000000008e-06, 'Cold': 3.981312000000001e-06}, {'Hot': 7.644119040000002e-07, 'Cold': 1.9906560000000005e-07}]

Viterbi2:  7.962624000000002e-07
path2:  ['Hot', 'Hot', 'Cold', 'Cold', 'Hot', 'Hot', 'Hot', 'Cold', 'Cold']
[{'Hot': 0.32000000000000006, 'Cold': 0.020000000000000004}, {'Hot': 0.07680000000000002, 'Cold': 0.009600000000000003}, {'Hot': 0.009216000000000002, 'Cold': 0.011520000000000002}, {'Hot': 0.0011059200000000003, 'Co

## Problem 3

In [27]:
import nltk 
from nltk.book import *
import pprint
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

In [97]:
def findlongest(text):
    #since we don't know how to convert the text object of nltk to string, we simply add all tokens togather to form a string.
    a = text.tokens
    text = ''
    for i in a:
        if i.isalpha():
            text = text + ' '+i
       
        else:
            text = text +i
    sents = sent_tokenizer.tokenize(text)
    lensent = []
    for i in sents:
        lensent.append(len(i.split()))
    longest = sents[lensent.index(max(lensent))]
    return longest

In [91]:
print('Moby Dick - the longest sentence:')
print(findlongest(text1))

Moby Dick - the longest sentence:
Though in many natural objects, whiteness refiningly enhances beauty, as if imparting some special virtue of its own, as in marbles, japonicas, and pearls; and though various nations have in some way recognised a certain royal preeminence in this hue; even the barbaric, grand old kings of Pegu placing the title" Lord of the White Elephants" above all their other magniloquent ascriptions of dominion; and the modern kings of Siam unfurling the same snow- white quadruped in the royal standard; and the Hanoverian flag bearing the one figure of a snow- white charger; and the great Austrian Empire, Caesarian, heir to overlording Rome, having for the imperial colour the same imperial hue; and though this pre- eminence in it applies to the human race itself, giving the white man ideal mastership over every dusky tribe; and though, besides, all this, whiteness has been even made significant of gladness, for among the Romans a white stone marked a joyful day; an

In [92]:
print('Sense and Sensibility - the longest sentence:')
print(findlongest(text2))

Sense and Sensibility - the longest sentence:
I am sure you will be glad to hear, as likewise dear Mrs. Jennings, I spent two happy hours with him yesterday afternoon, he would not hear of our parting, though earnestly did I, as I thought my duty required, urge him to it for prudence sake, and would have parted for ever on the spot, would he consent to it; but he said it should never be, he did not regard his mother' s anger, while he could have my affections; our prospects are not very bright, to be sure, but we must wait, and hope for the best; he will be ordained shortly; and should it ever be in your power to recommend him to any body that has a living to bestow, am very sure you will not forget us, and dear Mrs. Jennings too, trust she will speak a good word for us to Sir John, or Mr. Palmer, or any friend that may be able to assist us.-- Poor Anne was much to blame for what she did, but she did it for the best, so I say nothing; hope Mrs. Jennings won' t think it too much trouble

In [93]:
print('Inaugural Address Corpus - the longest sentence:')
print(findlongest(text4))

Inaugural Address Corpus - the longest sentence:
On this subject it might become me better to be silent or to speak with diffidence; but as something may be expected, the occasion, I hope, will be admitted as an apology if I venture to say that if a preference, upon principle, of a free republican government, formed upon long and serious reflection, after a diligent and impartial inquiry after truth; if an attachment to the Constitution of the United States, and a conscientious determination to support it until it shall be altered by the judgments and wishes of the people, expressed in the mode prescribed in it; if a respectful attention to the constitutions of the individual States and a constant caution and delicacy toward the State governments; if an equal and impartial regard to the rights, interest, honor, and happiness of all the States in the Union, without preference or regard to a northern or southern, an eastern or western, position, their various political opinions on unesse

In [94]:
print('Wall Street Journal - the longest sentence:')
print(findlongest(text7))

Wall Street Journal - the longest sentence:
But other than the fact that besuboru is played*-1 with a ball and a bat, it's unrecognizable: Fans politely return foul balls to stadium ushers; the strike zone expands depending on the size of the hitter; ties are permitted*-2-- even welcomed*-2-- since they honorably sidestep the shame of defeat; players must abide by strict rules of conduct even in their personal lives-- players for the Tokyo Giants, for example, must always wear ties when on the road.`` You Gotta Have Wa'' is the often amusing chronicle of how American ballplayers, rationed* to two per team, fare*T*-1 in Japan.


In [95]:
print('The Man Who Was Thursday - the longest sentence:')
print(findlongest(text9))

The Man Who Was Thursday - the longest sentence:
As I say," resumed the Professor, like a man toiling through heavy sand," the incident that has occurred to us and has led us to ask for information about the Marquis, is one which you may think it better to have narrated; but as it came in the way of Comrade Syme rather than me--" His words he seemed to be dragging out like words in an anthem; but Syme, who was watching, saw his long fingers rattle quickly on the edge of the crazy table.


Those are the longest sentence we found for those texts by using sent_tokenizer. The first letter of all of the sentences we found are capital letter. This means it only check the beginning sentence of every paragraphs and to find the longest one. And this make some mistakes on some text, like text2:Sense and Sensibility, the longest sentence it found is a paragraph(it may be the mistake we made).